In [1]:
# 패션 MNIST - 훈련세트, 테스트 세트
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [2]:
# 모델 생성 함수 - 층 추가 가능
def model_fn(a_layer = None):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(28,28))) # 입력층
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(100, activation='relu'))
    if a_layer:
        model.add(a_layer)

    model.add(keras.layers.Dense(10, activation='softmax'))

    return model

In [3]:
model = model_fn(keras.layers.Dropout(0.3))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
train_scaled= train_input / 255
test_scaled= test_input / 255

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 검증 세트 정확도가 떨어지기 직전 시점에 모델을 파일로 저장
# 과대 적합 직전에 모델을 파일로 저장한다.
checkpoint_cb = keras.callbacks.ModelCheckpoint("best-model.keras",save_best_only=True)


# 과대 접합 되기 직전 까지만 훈련하고 종료 - 조기 종료
# 검증 세트의 정확도가 2회 이상 올라가지 않으면 조기 종료(patience)
# restore_best_weights: 가장 최적의 가중치로 복구
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
model.fit(train_scaled, train_target, epochs=20,
validation_data=(test_scaled, test_target), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7428 - loss: 0.7277 - val_accuracy: 0.8400 - val_loss: 0.4513
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8435 - loss: 0.4341 - val_accuracy: 0.8553 - val_loss: 0.4003
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8563 - loss: 0.3939 - val_accuracy: 0.8606 - val_loss: 0.3790
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8622 - loss: 0.3765 - val_accuracy: 0.8647 - val_loss: 0.3700
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8691 - loss: 0.3580 - val_accuracy: 0.8750 - val_loss: 0.3576
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8727 - loss: 0.3471 - val_accuracy: 0.8770 - val_loss: 0.3480
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8795 - loss: 0.3307 - val_accuracy: 0.8760 - val_loss: 0.3533
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8782 - loss: 0.3248 - 

In [5]:
model = keras.models.load_model("best-model.keras")

In [6]:
model.evaluate(test_scaled, test_target)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 782us/step - accuracy: 0.8772 - loss: 0.3432


[0.34802156686782837, 0.8769999742507935]

In [7]:
# 조기 종료 시점
early_stopping_cb.stopped_epoch

7